In [ ]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import time

import sqlite3 # database for OLTP
import duckdb  # database for OLAP

In [ ]:
print("sqlite3 :", sqlite3.sqlite_version)
print("duckdb :", duckdb.__version__)

In [ ]:
# fixed seed
np.random.seed(1)

# scale
N = 2_000_000
users = 10_000
products = 1_000

In [ ]:
# sales sample
df = pd.DataFrame({
    "id"         : np.arange(N, dtype = np.int64),
    "user_id"    : np.random.randint(1, users + 1, size = N, dtype = np.int32),
    "product_id" : np.random.randint(1, products + 1, size = N, dtype = np.int32),
    "price"      : np.random.randint(100, 10000, size = N, dtype = np.int32),
    "quantity"   : np.random.randint(1, 6, size = N, dtype = np.int16),
    "timestamp"  : pd.to_datetime("2024-01-01") + pd.to_timedelta(np.random.randint(0, 60*60*24*30, size=N), unit="s")
})

In [ ]:
# product sample
prod = pd.DataFrame({
    "product_id" : np.arange(1, products + 1, dtype = np.int32),
    "category"   : np.random.randint(1, 21, size = products, dtype = np.int16)
})

In [ ]:
Path("data").mkdir(exist_ok = True)

# save data
df.to_parquet("data/sales.parquet")
prod.to_parquet("data/products.parquet")

len(df), len(prod), "saved to data/*.parquet"

In [ ]:
# sqlite3 setting

sqlite_path = "data/sales.db"

if os.path.exists(sqlite_path) : os.remove(sqlite_path)

conn_sqlite = sqlite3.connect(sqlite_path)
df.to_sql("sales", conn_sqlite, if_exists = "replace", index = False)
prod.to_sql("products", conn_sqlite, if_exists = "replace", index = False)

# index
cur = conn_sqlite.cursor()
cur.execute("CREATE INDEX idx_sales_id ON sales(id);")
cur.execute("CREATE INDEX idx_sales_user ON sales(user_id);")
cur.execute("CREATE INDEX idx_sales_product ON sales(product_id);")
cur.execute("CREATE INDEX idx_prod_id ON products(product_id);")
conn_sqlite.commit()

# check
cur.execute("SELECT count(*) FROM sales;")
print("SQLite sales rows : ", cur.fetchone()[0])
cur.execute("SELECT count(*) FROM products;")
print("SQLite products rows : ", cur.fetchone()[0])

In [ ]:
# duck_db setting

conn_duck = duckdb.connect()

conn_duck.execute("""
CREATE OR REPLACE TABLE sales AS SELECT * FROM read_parquet('data/sales.parquet');
""")
conn_duck.execute("""
CREATE OR REPLACE TABLE products AS SELECT * FROM read_parquet('data/products.parquet');
""")

# check
print("DuckDB sales rows:", conn_duck.execute("SELECT COUNT(*) FROM sales;").fetchone()[0])
print("DuckDB products rows:", conn_duck.execute("SELECT COUNT(*) FROM products;").fetchone()[0])

In [ ]:
def run_sqlite (query, fetch = 5) :
    t0 = time.perf_counter()      # time start
    cur = conn_sqlite.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    dt = time.perf_counter() - t0 # time end
    return dt, rows[:fetch]

def run_duck (query, fetch = 5) :
    t0 = time.perf_counter()      # time start
    rows = conn_duck.execute(query).fetchall()
    dt = time.perf_counter() - t0 # time end
    return dt, rows[:fetch]

def compare (title, q_sqlite, q_duck = None, fetch = 5) :
    if q_duck is None : q_duck = q_sqlite  # same query

    dt_s, head_s = run_sqlite(q_sqlite, fetch)
    dt_d, head_d = run_duck(q_duck, fetch)

    print(f"\n=== {title} ===")
    print(f"SQLite : {dt_s:.3f}s  | sample : {head_s[:2]}")
    print(f"DuckDB : {dt_d:.3f}s  | sample : {head_d[:2]}")

In [ ]:
# Task 1
# 단순 검색

q1 = "SELECT * FROM sales WHERE id = 123456;"
compare("OLTP : 단순 키 조회 (id)", q1)

In [ ]:
# Task 2
# 좁은 범위 탐색

q2 = """
SELECT * FROM sales
WHERE user_id = 777
ORDER BY timestamp DESC
LIMIT 50;
"""
compare("OLTP : 좁은 범위 조회 (user_id)", q2)

In [ ]:
# Task 3
# 제품별 매출, 상위 10개
# Group

q3 = """
SELECT product_id, SUM(price*quantity) AS total_sales
FROM sales
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 10;
"""
compare("OLAP : 제품별 총 매출 TOP10", q3)

In [ ]:
# Task 4
# 일자별 매출 집계

q4_sqlite = """
SELECT strftime('%Y-%m-%d', timestamp) AS d, SUM(price*quantity) AS revenue
FROM sales
GROUP BY d
ORDER BY d
LIMIT 30;
"""

# DuckDB는 date_trunc → cast
q4_duck = """
SELECT CAST(date_trunc('day', timestamp) AS DATE) AS d, SUM(price*quantity) AS revenue
FROM sales
GROUP BY d
ORDER BY d
LIMIT 30;
"""
compare("OLAP : 일자별 매출 집계", q4_sqlite, q4_duck)

In [ ]:
# Task 5
# 카테고리별 매출, 상위 5개
# Join

q5 = """
SELECT p.category, SUM(s.price*s.quantity) AS total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.category
ORDER BY total_sales DESC
LIMIT 5;
"""
compare("OLAP : 카테고리별 매출 TOP5", q5)

In [ ]:
# Task 6
# 사용자 별 매출, 상위 5개
# Group

q6 = """
SELECT user_id, SUM(price*quantity) AS total_spend, COUNT(*) AS cnt
FROM sales
GROUP BY user_id
ORDER BY total_spend DESC
LIMIT 10;
"""
compare("OLAP : 사용자별 매출 TOP10", q6)